# TIME SERIES

# National Stock Exchange of India.

Data link https://www.kaggle.com/datasets/minatverma/nse-stocks-data/data

Long short-term memory (LSTM)-
network is a recurrent neural network (RNN), aimed to deal with the vanishing gradient problem present in traditional RNNs

# Imports (import libraries)

In [ ]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
#import pandas_datareader as web
import datetime as dt

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

print ("The dependancies of the work")

In [16]:
stock_df = pd.read_csv("india_NSE.csv")

In [17]:
stock_df.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN
0,20MICRONS,EQ,37.80,37.80,36.15,36.85,37.40,37.05,27130,994657.90,2017-06-28,202,INE144J01027
1,3IINFOTECH,EQ,4.10,4.85,4.00,4.55,4.65,4.05,20157058,92148517.65,2017-06-28,7353,INE748C01020
2,3MINDIA,EQ,13425.15,13469.55,12920.00,13266.70,13300.00,13460.55,2290,30304923.35,2017-06-28,748,INE470A01017
3,63MOONS,EQ,61.00,61.90,60.35,61.00,61.10,60.65,27701,1689421.00,2017-06-28,437,INE111B01023
4,8KMILES,EQ,546.10,548.00,535.00,537.45,535.20,547.45,79722,43208620.05,2017-06-28,1866,INE650K01021


In [18]:
nse.tail()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN
846399,ZOTA,SM,221.0,221.00,214.00,217.00,217.0,220.0,14000,3041550.0,2017-12-29,14,INE358U01012
846400,ZUARI,EQ,562.0,562.00,549.55,550.80,552.0,556.8,105184,58163061.6,2017-12-29,2218,INE840M01016
846401,ZUARIGLOB,EQ,255.5,258.40,252.50,253.25,253.0,254.1,90067,22968100.6,2017-12-29,2001,INE217A01012
846402,ZYDUSWELL,EQ,1038.0,1068.00,1033.00,1043.20,1035.0,1023.1,28601,29938840.1,2017-12-29,1401,INE768C01010
846403,ZYLOG,EQ,4.3,5.15,4.30,4.65,4.6,4.4,732051,3495213.7,2017-12-29,845,INE225I01026


# Data preprocessing

In [19]:
stock_df.columns

Index(['SYMBOL', 'SERIES', 'OPEN', 'HIGH', 'LOW', 'CLOSE', 'LAST', 'PREVCLOSE',
       'TOTTRDQTY', 'TOTTRDVAL', 'TIMESTAMP', 'TOTALTRADES', 'ISIN'],
      dtype='object')

In [20]:
# convert TIMESTAMP to datetime instance
stock_df['TIMESTAMP'] = pd.to_datetime(stock_df.TIMESTAMP, dayfirst = True)
stock_df.set_index('TIMESTAMP', inplace = True)
stock_df['PER_DAILY_CHANGE'] = (stock_df.CLOSE - stock_df.OPEN)*100/stock_df.OPEN
stock_df.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,ISIN,PER_DAILY_CHANGE
TIMESTAMP,,,,,,,,,,,,,
2017-06-28,20MICRONS,EQ,37.80,37.80,36.15,36.85,37.40,37.05,27130,994657.90,202,INE144J01027,-2.513228
2017-06-28,3IINFOTECH,EQ,4.10,4.85,4.00,4.55,4.65,4.05,20157058,92148517.65,7353,INE748C01020,10.975610
2017-06-28,3MINDIA,EQ,13425.15,13469.55,12920.00,13266.70,13300.00,13460.55,2290,30304923.35,748,INE470A01017,-1.180248
2017-06-28,63MOONS,EQ,61.00,61.90,60.35,61.00,61.10,60.65,27701,1689421.00,437,INE111B01023,0.000000
2017-06-28,8KMILES,EQ,546.10,548.00,535.00,537.45,535.20,547.45,79722,43208620.05,1866,INE650K01021,-1.583959


In [21]:
#Lets check how many different symbols are present in the dataset
unique_symb = stock_df['SYMBOL'].unique()
print("total number of unique symbols is ", len(unique_symb))

#let's print top 10 different symbols
unique_symb[:10]

total number of unique symbols is  2037


array(['20MICRONS', '3IINFOTECH', '3MINDIA', '63MOONS', '8KMILES',
       'A2ZINFRA', 'AARTIDRUGS', 'AARTIIND', 'AARVEEDEN', 'ABAN'],
      dtype=object)

In [22]:
# total number of datapoints
stock_df.shape

(846404, 13)

846404 rows and 13 colummns

In [23]:
#say if we are interested in predicting prices of Banks only
bank_stock = stock_df[stock_df['SYMBOL'].str.contains("BANK")]
bank_stock.head()

,SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TOTALTRADES,ISIN,PER_DAILY_CHANGE
TIMESTAMP,,,,,,,,,,,,,
2017-06-28,ANDHRABANK,EQ,52.60,54.25,52.40,53.95,54.05,53.45,3191618,1.708973e+08,7202,INE434A01013,2.566540
2017-06-28,AXISBANK,EQ,493.80,500.35,490.05,492.00,496.45,492.85,12691859,6.260657e+09,76061,INE238A01034,-0.364520
2017-06-28,BANKBARODA,EQ,153.00,156.40,152.20,155.60,155.90,154.85,6999255,1.083748e+09,41604,INE028A01039,1.699346
2017-06-28,BANKBEES,EQ,2341.89,2355.72,2335.91,2354.88,2355.00,2347.31,8106,1.904236e+07,318,INF732E01078,0.554680
2017-06-28,BANKINDIA,EQ,132.00,137.65,131.35,136.75,136.55,133.55,4789755,6.463776e+08,22220,INE084A01016,3.598485
